In [1]:
import cv2
import numpy as np
import helpers

def showImage(img):

    #define the screen resulation
    screen_res = 1280, 720
    scale_width = screen_res[0] / img.shape[1]
    scale_height = screen_res[1] / img.shape[0]
    scale = min(scale_width, scale_height)

    #resized window width and height
    window_width = int(img.shape[1] * scale)
    window_height = int(img.shape[0] * scale)

    #cv2.WINDOW_NORMAL makes the output window resizealbe
    cv2.namedWindow('Resized Window', cv2.WINDOW_NORMAL)

    #resize the window according to the screen resolution
    cv2.resizeWindow('Resized Window', window_width, window_height)

    cv2.imshow('Resized Window', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [2]:
image = cv2.imread('puzz1.jpg',0)
outerBox = np.ndarray(image.shape)

<a href='https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/'>Gaussian Blur</a>

In [3]:
image = cv2.GaussianBlur(image,(11,11),0)

<a href="https://pyimagesearch.com/2021/05/12/adaptive-thresholding-with-opencv-cv2-adaptivethreshold/">Adaptive Thresholding</a>

In [4]:
outerBox = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 2)

In [5]:
outerBox = cv2.bitwise_not(outerBox)

<a href='https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html'>Dilation</a>

In [6]:
outerBox = cv2.dilate(outerBox, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)),iterations=4)

In [8]:
contours, heirarchy = cv2.findContours(outerBox, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [11]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4,random_state=0).fit(contours[0][:,0,:])

In [42]:
grid_corners = kmeans.cluster_centers_
image_corners_dict = {'topLeft':(0,0), 'topRight':(image.shape[1],0), 'bottomLeft':(0,image.shape[0]), 'bottomRight':(image.shape[1],image.shape[0])}
grid_corners_dict = {r: grid_corners[np.argmin([np.linalg.norm(i-np.array(k)) for i in grid_corners])] for (r,k) in image_corners_dict.items()}

In [58]:
top_x_spacing = np.linspace(grid_corners_dict['topLeft'], grid_corners_dict['topRight'],num = 10)
bottom_x_spacing = np.linspace(grid_corners_dict['bottomLeft'], grid_corners_dict['bottomRight'],num = 10)
left_y_spacing = np.linspace(grid_corners_dict['topLeft'], grid_corners_dict['bottomLeft'],num = 10)
right_y_spacing = np.linspace(grid_corners_dict['topRight'], grid_corners_dict['bottomRight'],num = 10)


In [55]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

In [60]:
def get_intersetion(vert_line_idx, hor_line_idx):
    return line_intersection((top_x_spacing[vert_line_idx],bottom_x_spacing[vert_line_idx]),
                            (left_y_spacing[hor_line_idx],right_y_spacing[hor_line_idx]))

In [64]:
def get_gridbox_corners_at(point):
    return {'topLeft': get_intersetion(point[0],point[1]),
            'topRight': get_intersetion(point[0]+1,point[1]),
            'bottomLeft': get_intersetion(point[0],point[1]+1),
            'bottomRight': get_intersetion(point[0]+1,point[1]+1)}

{'topLeft': (989.8765432098766, 998.942386831276),
 'topRight': (1112.111111111111, 998.9629629629632),
 'bottomLeft': (989.7777777777778, 1121.0),
 'bottomRight': (1112.0, 1121.0)}